In [8]:
'''
    resnet 3D CNN + kinetics-400 pretrained + hmdb51 training
    Reference paper: Can Spatiotemporal 3D CNNs Retrace the History of 2D CNNs and ImageNet?
'''

import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchsummaryX import summary
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import pretrained_model as model
import os
import time
import copy
import warnings
import utils

In [4]:
warnings.filterwarnings("ignore")

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model_ft = model.get_model().to(device)
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters())
log_name = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()))
writer = SummaryWriter(os.path.join('log/', log_name))

print(model_ft)
summary(model_ft, torch.zeros(1, 3, 8, 112, 112).to(device))

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_stem.Conv3d_0,"[3, 64, 3, 7, 7]","[1, 64, 8, 56, 56]",28224.0,7.080837e+08
1_stem.BatchNorm3d_1,[64],"[1, 64, 8, 56, 56]",128.0,6.400000e+01
2_stem.ReLU_2,-,"[1, 64, 8, 56, 56]",NaN,NaN
3_layer1.0.conv1.Conv3DSimple_0,"[64, 64, 3, 3, 3]","[1, 64, 8, 56, 56]",110592.0,2.774532e+09
4_layer1.0.conv1.BatchNorm3d_1,[64],"[1, 64, 8, 56, 56]",128.0,6.400000e+01
5_layer1.0.conv1.ReLU_2,-,"[1, 64, 8, 56, 56]",NaN,NaN
6_layer1.0.conv2.Conv3DSimple_0,"[64, 64, 3, 3, 3]","[1, 64, 8, 56, 56]",110592.0,2.774532e+09
7_layer1.0.conv2.BatchNorm3d_1,[64],"[1, 64, 8, 56, 56]",128.0,6.400000e+01
8_layer1.0.ReLU_relu,-,"[1, 64, 8, 56, 56]",NaN,NaN


In [10]:
best_val_acc = 0.0
best_val_model = None
EPOCH = 10
BSZ = 16

transform = transforms.Compose([utils.RandomResizedCropVideo(112), utils.ToTensorVideo()])

data_root = '../../Data/HMDB'

train_loader = DataLoader(datasets.HMDB51(root = os.path.join(data_root, 'hmdb51'), 
                                          annotation_path = os.path.join(data_root,'splits'),
                                          frames_per_clip = 8, fold = 1, train = True, 
                                          transform = transform), 
                          batch_size = BSZ, shuffle = True, num_workers = 4)

# val_loader = DataLoader(datasets.HMDB51(root = os.path.join(data_root, 'hmdb51'), 
#                                         annotation_path = os.path.join(data_root, 'splits'),
#                                         frames_per_clip = 8, fold = 2, train = False, 
#                                         transform = transform), 
#                         batch_size = BSZ, shuffle = True, num_workers = 4)

# test_loader = DataLoader(datasets.HMDB51(root = os.path.join(data_root, 'hmdb51'), 
#                                          annotation_path = os.path.join(data_root, 'splits'),
#                                          frames_per_clip = 8, fold = 3, train = False, 
#                                          transform = transform), 
#                          batch_size = BSZ, shuffle = True, num_workers = 4)



sample_size = next(iter(train_loader))[0].size()
assert sample_size == torch.Size([BSZ, 3, 8, 112, 112]), 'sample_size is {}'.format(sample_size)

100.0%


In [11]:
for epoch in range(EPOCH):
    
    start_time = time.strftime("%H:%M:%S", time.localtime(time.time()))
    print(f'epoch {epoch} | start time {start_time}')

    train_loss, train_acc = model.train(model_ft, train_loader, criterion, optimizer, epoch, writer, device)
    val_loss, val_acc = model.evaluate(model_ft, val_loader, criterion, epoch, writer, device)

    print(f'train loss {train_loss:03f} | train accuracy {train_acc:03f}')
    print(f'val loss {val_loss:03f} | val accuracy {val_acc:03f}\n')

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_val_model = copy.deepcopy(model_ft.state_dict())

model_ft.load_state_dict(best_val_model)
test_loss, test_acc = model.evaluate(model_ft, test_loader, criterion, EPOCH+1, writer, device)
print(f'test_loss {test_loss:03f} | test acc {test_acc:03f}')



epoch 0 | start time 16:11:28
	train epoch 0 | 0/19998 | duration 0m:0s | loss 3.832685 | accuracy 0.000000
	train epoch 0 | 1/19998 | duration 0m:0s | loss 3.812154 | accuracy 0.125000
	train epoch 0 | 2/19998 | duration 0m:0s | loss 3.920867 | accuracy 0.000000
	train epoch 0 | 3/19998 | duration 0m:0s | loss 3.852073 | accuracy 0.062500
	train epoch 0 | 4/19998 | duration 0m:0s | loss 3.904992 | accuracy 0.125000
	train epoch 0 | 5/19998 | duration 0m:0s | loss 3.878559 | accuracy 0.125000
	train epoch 0 | 6/19998 | duration 0m:0s | loss 3.869922 | accuracy 0.000000
	train epoch 0 | 7/19998 | duration 0m:0s | loss 3.875477 | accuracy 0.125000
	train epoch 0 | 8/19998 | duration 0m:0s | loss 3.914709 | accuracy 0.000000
	train epoch 0 | 9/19998 | duration 0m:0s | loss 3.814213 | accuracy 0.187500
	train epoch 0 | 10/19998 | duration 0m:0s | loss 3.822583 | accuracy 0.187500
	train epoch 0 | 11/19998 | duration 0m:0s | loss 3.832159 | accuracy 0.000000
	train epoch 0 | 12/19998 | dura

	train epoch 0 | 104/19998 | duration 0m:0s | loss 4.043955 | accuracy 0.000000
	train epoch 0 | 105/19998 | duration 0m:0s | loss 3.867981 | accuracy 0.000000
	train epoch 0 | 106/19998 | duration 0m:0s | loss 3.799778 | accuracy 0.000000
	train epoch 0 | 107/19998 | duration 0m:0s | loss 3.806776 | accuracy 0.062500
	train epoch 0 | 108/19998 | duration 0m:0s | loss 3.804651 | accuracy 0.062500
	train epoch 0 | 109/19998 | duration 0m:0s | loss 3.688603 | accuracy 0.187500
	train epoch 0 | 110/19998 | duration 0m:0s | loss 3.303044 | accuracy 0.312500
	train epoch 0 | 111/19998 | duration 0m:0s | loss 3.825871 | accuracy 0.125000
	train epoch 0 | 112/19998 | duration 0m:0s | loss 3.689234 | accuracy 0.125000
	train epoch 0 | 113/19998 | duration 0m:0s | loss 3.779493 | accuracy 0.062500
	train epoch 0 | 114/19998 | duration 0m:0s | loss 3.837780 | accuracy 0.000000
	train epoch 0 | 115/19998 | duration 0m:0s | loss 3.941573 | accuracy 0.000000
	train epoch 0 | 116/19998 | duration 0m

	train epoch 0 | 207/19998 | duration 0m:0s | loss 3.820091 | accuracy 0.062500
	train epoch 0 | 208/19998 | duration 0m:0s | loss 3.941035 | accuracy 0.125000
	train epoch 0 | 209/19998 | duration 0m:0s | loss 3.816237 | accuracy 0.062500
	train epoch 0 | 210/19998 | duration 0m:0s | loss 3.983354 | accuracy 0.062500
	train epoch 0 | 211/19998 | duration 0m:0s | loss 3.677366 | accuracy 0.062500
	train epoch 0 | 212/19998 | duration 0m:0s | loss 3.908607 | accuracy 0.062500
	train epoch 0 | 213/19998 | duration 0m:0s | loss 3.725478 | accuracy 0.062500
	train epoch 0 | 214/19998 | duration 0m:0s | loss 3.771262 | accuracy 0.062500
	train epoch 0 | 215/19998 | duration 0m:0s | loss 3.724787 | accuracy 0.125000
	train epoch 0 | 216/19998 | duration 0m:0s | loss 4.057250 | accuracy 0.000000
	train epoch 0 | 217/19998 | duration 0m:0s | loss 3.739351 | accuracy 0.062500
	train epoch 0 | 218/19998 | duration 0m:0s | loss 3.669828 | accuracy 0.000000
	train epoch 0 | 219/19998 | duration 0m

	train epoch 0 | 310/19998 | duration 0m:0s | loss 3.700763 | accuracy 0.000000
	train epoch 0 | 311/19998 | duration 0m:0s | loss 3.965553 | accuracy 0.000000
	train epoch 0 | 312/19998 | duration 0m:0s | loss 3.949599 | accuracy 0.000000
	train epoch 0 | 313/19998 | duration 0m:0s | loss 3.844572 | accuracy 0.000000
	train epoch 0 | 314/19998 | duration 0m:0s | loss 3.609790 | accuracy 0.125000
	train epoch 0 | 315/19998 | duration 0m:0s | loss 3.718147 | accuracy 0.000000
	train epoch 0 | 316/19998 | duration 0m:0s | loss 3.749156 | accuracy 0.062500
	train epoch 0 | 317/19998 | duration 0m:0s | loss 3.676732 | accuracy 0.125000
	train epoch 0 | 318/19998 | duration 0m:0s | loss 3.870896 | accuracy 0.000000
	train epoch 0 | 319/19998 | duration 0m:0s | loss 3.683811 | accuracy 0.062500
	train epoch 0 | 320/19998 | duration 0m:0s | loss 3.708071 | accuracy 0.125000
	train epoch 0 | 321/19998 | duration 0m:0s | loss 3.479856 | accuracy 0.187500
	train epoch 0 | 322/19998 | duration 0m

	train epoch 0 | 413/19998 | duration 0m:0s | loss 4.063847 | accuracy 0.062500
	train epoch 0 | 414/19998 | duration 0m:0s | loss 4.083677 | accuracy 0.000000
	train epoch 0 | 415/19998 | duration 0m:0s | loss 3.634018 | accuracy 0.187500
	train epoch 0 | 416/19998 | duration 0m:0s | loss 3.647251 | accuracy 0.000000
	train epoch 0 | 417/19998 | duration 0m:0s | loss 3.727756 | accuracy 0.062500
	train epoch 0 | 418/19998 | duration 0m:0s | loss 3.890863 | accuracy 0.000000
	train epoch 0 | 419/19998 | duration 0m:0s | loss 3.911141 | accuracy 0.000000
	train epoch 0 | 420/19998 | duration 0m:0s | loss 3.949013 | accuracy 0.062500
	train epoch 0 | 421/19998 | duration 0m:0s | loss 3.685788 | accuracy 0.062500
	train epoch 0 | 422/19998 | duration 0m:0s | loss 3.924021 | accuracy 0.062500
	train epoch 0 | 423/19998 | duration 0m:0s | loss 3.867464 | accuracy 0.000000
	train epoch 0 | 424/19998 | duration 0m:0s | loss 3.760839 | accuracy 0.187500
	train epoch 0 | 425/19998 | duration 0m

...

	train epoch 0 | 19607/19998 | duration 0m:0s | loss 1.055853 | accuracy 0.812500
	train epoch 0 | 19608/19998 | duration 0m:0s | loss 1.306256 | accuracy 0.562500
	train epoch 0 | 19609/19998 | duration 0m:0s | loss 0.585034 | accuracy 0.875000
	train epoch 0 | 19610/19998 | duration 0m:0s | loss 1.160683 | accuracy 0.625000
	train epoch 0 | 19611/19998 | duration 0m:0s | loss 1.083996 | accuracy 0.687500
	train epoch 0 | 19612/19998 | duration 0m:0s | loss 1.967207 | accuracy 0.437500
	train epoch 0 | 19613/19998 | duration 0m:0s | loss 1.285827 | accuracy 0.500000
	train epoch 0 | 19614/19998 | duration 0m:0s | loss 0.829065 | accuracy 0.812500
	train epoch 0 | 19615/19998 | duration 0m:0s | loss 2.034417 | accuracy 0.562500
	train epoch 0 | 19616/19998 | duration 0m:0s | loss 1.224939 | accuracy 0.687500
	train epoch 0 | 19617/19998 | duration 0m:0s | loss 0.874838 | accuracy 0.687500
	train epoch 0 | 19618/19998 | duration 0m:0s | loss 0.993224 | accuracy 0.812500
	train epoch 0 |

	train epoch 0 | 19707/19998 | duration 0m:0s | loss 1.164826 | accuracy 0.625000
	train epoch 0 | 19708/19998 | duration 0m:0s | loss 0.599255 | accuracy 0.937500
	train epoch 0 | 19709/19998 | duration 0m:0s | loss 0.847127 | accuracy 0.812500
	train epoch 0 | 19710/19998 | duration 0m:0s | loss 1.572975 | accuracy 0.500000
	train epoch 0 | 19711/19998 | duration 0m:0s | loss 1.366894 | accuracy 0.687500
	train epoch 0 | 19712/19998 | duration 0m:0s | loss 0.730127 | accuracy 0.812500
	train epoch 0 | 19713/19998 | duration 0m:0s | loss 1.029943 | accuracy 0.625000
	train epoch 0 | 19714/19998 | duration 0m:0s | loss 0.805546 | accuracy 0.750000
	train epoch 0 | 19715/19998 | duration 0m:0s | loss 0.864863 | accuracy 0.812500
	train epoch 0 | 19716/19998 | duration 0m:0s | loss 1.512973 | accuracy 0.687500
	train epoch 0 | 19717/19998 | duration 0m:0s | loss 1.505246 | accuracy 0.625000
	train epoch 0 | 19718/19998 | duration 0m:0s | loss 0.821054 | accuracy 0.812500
	train epoch 0 |

	train epoch 0 | 19807/19998 | duration 0m:0s | loss 1.303202 | accuracy 0.562500
	train epoch 0 | 19808/19998 | duration 0m:0s | loss 1.589226 | accuracy 0.562500
	train epoch 0 | 19809/19998 | duration 0m:0s | loss 2.352371 | accuracy 0.375000
	train epoch 0 | 19810/19998 | duration 0m:0s | loss 1.169611 | accuracy 0.687500
	train epoch 0 | 19811/19998 | duration 0m:0s | loss 0.538496 | accuracy 0.937500
	train epoch 0 | 19812/19998 | duration 0m:0s | loss 1.219739 | accuracy 0.687500
	train epoch 0 | 19813/19998 | duration 0m:0s | loss 1.103197 | accuracy 0.687500
	train epoch 0 | 19814/19998 | duration 0m:0s | loss 1.121245 | accuracy 0.750000
	train epoch 0 | 19815/19998 | duration 0m:0s | loss 0.361857 | accuracy 0.937500
	train epoch 0 | 19816/19998 | duration 0m:0s | loss 1.613574 | accuracy 0.625000
	train epoch 0 | 19817/19998 | duration 0m:0s | loss 1.045477 | accuracy 0.687500
	train epoch 0 | 19818/19998 | duration 0m:0s | loss 1.123295 | accuracy 0.750000
	train epoch 0 |

	train epoch 0 | 19907/19998 | duration 0m:0s | loss 1.541425 | accuracy 0.437500
	train epoch 0 | 19908/19998 | duration 0m:0s | loss 1.952996 | accuracy 0.375000
	train epoch 0 | 19909/19998 | duration 0m:0s | loss 1.690962 | accuracy 0.562500
	train epoch 0 | 19910/19998 | duration 0m:0s | loss 1.552525 | accuracy 0.625000
	train epoch 0 | 19911/19998 | duration 0m:0s | loss 0.943606 | accuracy 0.812500
	train epoch 0 | 19912/19998 | duration 0m:0s | loss 0.722255 | accuracy 0.812500
	train epoch 0 | 19913/19998 | duration 0m:0s | loss 0.536533 | accuracy 0.812500
	train epoch 0 | 19914/19998 | duration 0m:0s | loss 0.809679 | accuracy 0.750000
	train epoch 0 | 19915/19998 | duration 0m:0s | loss 1.444860 | accuracy 0.625000
	train epoch 0 | 19916/19998 | duration 0m:0s | loss 1.119786 | accuracy 0.625000
	train epoch 0 | 19917/19998 | duration 0m:0s | loss 1.225183 | accuracy 0.750000
	train epoch 0 | 19918/19998 | duration 0m:0s | loss 0.925316 | accuracy 0.750000
	train epoch 0 |

TypeError: only integer tensors of a single element can be converted to an index